In [1]:
from simulate_game import simulate_game
from competitive_sudoku.sudoku import load_sudoku

import importlib
import sys
import io
import os
import pandas as pd
import itertools

from tqdm import tqdm

In [2]:
# Winner 0 if it was a draw, 1 if player 1 won, 2 if player 2 won
# Reason:
# "regular" if it was played till the end
# "taboo" if a taboo move was played
# "invalid" if invalid move was played
# "illegal" if illegal move was played
# "no move" if no move was supplied


def run_simulation(first_player: str, second_player: str, board_name: str, time: float, return_text = False) -> set:
    player1 = importlib.import_module(first_player + '.sudokuai').SudokuAI()
    player2 = importlib.import_module(second_player + '.sudokuai').SudokuAI()
    player1.player_number = 1
    player2.player_number = 2
    
    # We do not check for specific ai, just give it to all of them
    player1.solve_sudoku_path = 'bin\\solve_sudoku.exe'
    player2.solve_sudoku_path = 'bin\\solve_sudoku.exe'
    
    board = load_sudoku(board_name)
    
    # Note for future, we do not clean up files
    
    old_stdout = sys.stdout
    sys.stdout = buffer = io.StringIO()
    
    simulate_game(board, player1, player2, solve_sudoku_path='bin\\solve_sudoku.exe', calculation_time=time)
    
    sys.stdout = old_stdout
    result = buffer.getvalue()
    
    last_line = result.splitlines()[-1]
    splitted_line = last_line.split(".")
    
    if len(splitted_line) == 2:
        winner = {"The game ends in a draw": 0, "Player 1 wins the game":1, 
                  "Player 2 wins the game":2}[splitted_line[0]]
        reason = "regular"
    
    else:
        winner = {" The game ends in a draw": 0, " Player 1 wins the game":1, 
                  " Player 2 wins the game":2}[splitted_line[1]]
        reason = {"taboo move": "taboo", "valid move": "invalid", "legal move": "illegal", 
                  "s supplied": "no move"}[splitted_line[0][-10:]]
    
    if not return_text:
        return winner, reason
    
    else:
        return winner, reason, result

In [3]:
opponents = ["random_player", "greedy_player"]
boards = os.listdir("boards")
times = [0.1, 0.5, 1, 5]
starting = [True, False]

In [4]:
testing = "team41_A1"
df = pd.DataFrame(columns = ["opponent", "board", "time", "starting", "result", "reason"])
counter = 0

for setup in tqdm(list(itertools.product(opponents, boards, times, starting))):
    if setup[3]:
        winner, reason = run_simulation(testing, setup[0], f'boards//{setup[1]}', setup[2])
        result = {0:0, 1:1, 2:-1}[winner]
    
    else:
        winner, reason = run_simulation(setup[0], testing, f'boards//{setup[1]}', setup[2])
        result = {0: 0, 1:-1, 2:1}[winner]
    
    new_row = pd.DataFrame({"opponent":setup[0][:-7], "board": setup[1][:-4], "time": setup[2], 
                            "starting": setup[3], "result": result, "reason": reason}, index=[0])
    
    df = pd.concat([df, new_row]).reset_index(drop = True)
    
    counter += 1
    if counter % 10 == 0:
        df.to_csv(f'testing results//{testing}.csv', index = False)

df.to_csv(f'testing results//{testing}.csv', index = False)

100%|██████████████████████████████████████████████████████████████████████████████| 192/192 [2:40:44<00:00, 50.23s/it]


In [14]:
df = pd.DataFrame(columns = ["opponent", "board", "time", "starting"])

for setup in tqdm(list(itertools.product(opponents, boards, times, starting))):
    new_row = pd.DataFrame({"opponent":setup[0][:-7], "board": setup[1][:-4], "time": setup[2], 
                            "starting": setup[3]}, index=[0])
    
    df = pd.concat([df, new_row]).reset_index(drop = True)
    
df.to_csv(f'testing results//base.csv', index = False)

100%|███████████████████████████████████████████████████████████████████████████████| 192/192 [00:00<00:00, 994.64it/s]
